In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import sys
import codecs
import mpld3

In [2]:
# initially we need document folders and document id's
path_to_dataset = '/home/madhi/Documents/python programs/neuralnetworks/fp/Reuters21578-Apte-115Cat/training'
folder_list = os.listdir(path_to_dataset)

In [3]:
#make a dictionary of document category and documents contain in the folder
cat_and_docs = [] #list
document_folder = [] #list
for docs_folder in folder_list:
    folder_path = path_to_dataset + "/" + docs_folder
    list_of_documents = os.listdir(folder_path)
    for real_docs in list_of_documents:
        document_folder.append(real_docs)
    #append folder_name and docs into the same list
    cat_and_docs.append((docs_folder,document_folder))
    #empty the document_folder list
    document_folder = []
    
print(cat_and_docs[1]) #the list is in format category and number of docs in that list

('palladium', ['0001792', '0001862'])


In [4]:
#make into dictionary
cate = [] 
documents = []

for category,docs in cat_and_docs:
    cate.append(category)
    documents.append(docs)
    
#convert into dictionary
total_dataset = dict(zip(cate,documents))

In [5]:
len(total_dataset)

116

In [6]:
feature_df = pd.read_csv('features.txt',sep='\t',header=None,names=['doc_id','features'])
feature_df.head()

,doc_id,features
0,0,"alfonsin,allsuite,arnott,bonded,briefed,bulax,..."
1,0,"dependent,cellulosa,allentown,bases,ergenc,err..."
2,0,"cellulosa,emphasizing,edina,celsius,dependent,..."
3,0,"edina,cellulosa,allentown,emphasizing,celsius,..."
4,0,"downed,ecologists,edina,emphasizing,cellulosa,..."


In [7]:
doc_category = feature_df['doc_id']
features = feature_df['features']
#convert into pandas Series
features_id = pd.Series(doc_category)
features_series = pd.Series(features)

print(features_id.shape,features_series.shape)

(11475,) (11475,)


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
#in features_series list all features is considered as a single string
#we need to sepearate the words and have to apply tf-idf vectorizer and ngram to each words
def document_category(features_id):
    docs_category = []
    for ids in features_id:
        docs_category.append(ids)
    return docs_category

def separate_features_the_series(feature_series):
    main_feature_series = []
    for features in feature_series:
        data_features = features.replace(","," ")
        main_feature_series.append(data_features)
    return main_feature_series

docs_id = document_category(features_id)
features_splitted = separate_features_the_series(features_series)
print(len(docs_id),len(features_splitted))

11475 11475


In [9]:
tfidf_vectorizer = TfidfVectorizer(max_df=1.0,max_features=20000,use_idf=True,ngram_range=(1,3))
%time  tfidf_matrix = tfidf_vectorizer.fit_transform(features_splitted)
print(tfidf_matrix.shape)

CPU times: user 660 ms, sys: 4 ms, total: 664 ms
Wall time: 802 ms
(11475, 14240)


In [10]:
tfidf_matrix

<11475x14240 sparse matrix of type '<class 'numpy.float64'>'
	with 309825 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

dist = 1-cosine_similarity(tfidf_matrix)

In [12]:
dist

array([[ 8.88178420e-16,  1.00000000e+00,  1.00000000e+00, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00, -6.66133815e-16,  9.30546629e-01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  9.30546629e-01,  0.00000000e+00, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       ...,
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
        -8.88178420e-16,  9.72391357e-01,  9.75894543e-01],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
         9.72391357e-01,  1.11022302e-16,  9.64164476e-01],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
         9.75894543e-01,  9.64164476e-01,  8.88178420e-16]])

In [13]:
len(cate)

116

In [14]:
len(documents)

116

In [15]:
from sklearn.externals import joblib
from sklearn.cluster import KMeans

num_clusters = 5
km = KMeans(n_clusters = num_clusters)

%time km.fit(tfidf_matrix)
#dump the clusters for later user
#joblib.dump(km,'clusters.pkl')

CPU times: user 8.56 s, sys: 96 ms, total: 8.66 s
Wall time: 8.9 s


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [16]:
#load the cluster
cl_load = joblib.load('clusters.pkl')
def_clusters = cl_load.labels_.tolist()
print(def_clusters[:96]) #just print to see the clusters

[1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 2, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 2, 3, 1, 1, 1, 2, 1, 2]


In [17]:
#convert features_splitted into a Series Pandas
document = {'category':docs_id,'content':features_splitted ,'clusters':def_clusters}
frame = pd.DataFrame(document,index=[def_clusters],columns=['category','content','clusters'])

In [18]:
frame['clusters'].value_counts()

1    7605
3    1290
2    1035
4     960
0     585
Name: clusters, dtype: int64

In [19]:
frame.iloc[9658:9668] #the less number of clusters the more number of same category documents belongs to same clusters

,category,content,clusters
1,87,dynalectron athletic elections bhputah denwa c...,1
3,87,detected conservatism bonn decided converts da...,3
1,87,coopervision clercq chatted enfield employee e...,1
1,87,cbl dwindled educate danger academics amnount ...,1
3,87,athletic alatenn cry counties arbitrary compac...,3
1,87,clercq discuss abating august completion brant...,1
3,87,athletic bale cole darman audit applications b...,3
1,87,ameritrust clipbecause airframe dnzr column be...,1
1,87,cbl breathing ecuadoreans culver dur clock cru...,1
1,87,cbl csx bni awaits creditcard coop creates dec...,1


In [20]:
grouped = frame['category'].groupby(frame['clusters'])
grouped.mean() 

clusters
0    51.921368
1    48.746614
2    47.077295
3    48.712403
4    48.685417
Name: category, dtype: float64

cluster 2 have lowest rank which indicates the document is being pretty good classified
cluster 1,3,4 are approximately equal to each other , document classified is average
cluster 0 have highest rank which some documents are unnecessary classified into that document

In [43]:
frame = frame.drop(['document_id','documents'],axis=1)

In [55]:
#convert document into series
series_docs = pd.Series(documents)
individual_docs = [] #to store individual documents
for document in series_docs:
    for indi_docs in document:
        individual_docs.append(indi_docs)
print(len(individual_docs))

11476


In [78]:
series_indi_docs = pd.Series(individual_docs)
series_indi_docs = series_indi_docs[:-1]
print(series_indi_docs.shape) #the shape of frame is (11475,4)

(11475,)


In [79]:
frame.head()

,category,content,clusters,document
1,0,alfonsin allsuite arnott bonded briefed bulax ...,1,NaN
1,0,dependent cellulosa allentown bases ergenc err...,1,NaN
1,0,cellulosa emphasizing edina celsius dependent ...,1,NaN
2,0,edina cellulosa allentown emphasizing celsius ...,2,NaN
1,0,downed ecologists edina emphasizing cellulosa ...,1,NaN


In [80]:
frame['document'] = series_indi_docs.values

In [82]:
frame.tail()

,category,content,clusters,document
0,113,aaa abates abdelrahim abdulaziz abf ability aa...,0,0005167
0,114,ababa aaminus aare abdul aar abdulaziz aarnoud...,0,0004782
0,114,aaa abdul abattoirs aas aaplus abnnas abandonm...,0,0005167
0,115,aaarated aaix abandon abbreviated abandoning a...,0,0004556
1,115,abf aachener abha aaminus abdulrahim abev abso...,1,0001973


In [124]:
from __future__ import print_function
terms = tfidf_vectorizer.get_feature_names()
#print(len(terms))  14,240 terms
important_terms = [] #to store important terms
important_terms_stemmed = [] #to store important terms stemmed
for terms in features_series:
    #replace the comma by space for word sepearation
    data = terms.replace(","," ")
    words_splitted = data.split()
    for words in words_splitted:
        important_terms.append(words)
print(len(important_terms))
vocab_frame = pd.DataFrame({'words': important_terms})

114750
